In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PAK TATANG FC****

pak tatang adalah presiden club dari tatang fc. sedang mencari pelatih dan pemain baru untuk club nya. kita di beri data seperti dibawah. kita disuruh mencari tipe pelatih dan pemain, agar pak tatang bisa menjadikan tatang fc, menjadi club terkuat di galaxy bima sakti. oke lebay maaf.

In [ ]:
dataset = pd.read_csv("/kaggle/input/prediksi-kemenangan-sepakbola/dataset.csv")

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().values.any()

In [ ]:
dataset["Pelatih"].value_counts()

# **Kita akan lihat persentase kemenangan pelatih dari tipe melatih**
Kalo anak MOBA bilang semua hero itu hebat, tergantung pilot. begitu juga club tatang fc, memerlukan pilot atau pelatih yang handal untuk menahkodai tatang fc

In [ ]:
tipe_pelatih =  dataset.loc[dataset['Menang'] == "Ya", 'Pelatih']

In [ ]:
tipe_pelatih.value_counts()

hitung persentase kemenangan dari total laga yang menghasilkan kemenangan

In [ ]:
pengertian = dataset.loc[dataset['Pelatih'] == "Pengetian"]
Menekan = dataset.loc[dataset['Pelatih'] == "Menekan"]

In [ ]:
p_menang = len(pengertian.loc[pengertian["Menang"] == "Ya"])/len(pengertian["Menang"])
m_menang = len(Menekan.loc[Menekan["Menang"] == "Ya"])/len(Menekan["Menang"])

In [ ]:
print("Tipe Pengertian : Match ",len(pengertian["Menang"]),"\nMenang : ",len(pengertian.loc[pengertian["Menang"] == "Ya"]),"\nKalah : ",len(pengertian.loc[pengertian["Menang"] == "Tidak"]))
print("Tipe Menekan : Match ",len(Menekan["Menang"]),"\nMenang : ",len(Menekan.loc[Menekan["Menang"] == "Ya"]),"\nKalah : ",len(Menekan.loc[Menekan["Menang"] == "Tidak"]))
print("Persentase Tipe Pengertian: {0:.2f} %".format(100 * p_menang))  
print("Persentase Tipe Menekan: {0:.2f} %".format(100 * m_menang))  

Dari data diatas, terlihat Pelatih Tipe Pengertian lebih unggul, mungkin dikarenakan faktor match yang lebih banyak dimainkan oleh pelatih pengertian dibandingkan pelatih tipe menekan.

**Dari pelatih sudah, sekarang ke pemain,Kita akan prediksi kemenangan**

Ubah datanya menjadi int dulu, biar gampang

In [ ]:
pelatih = {"Pengetian" : 0, "Menekan" : 1}
main = {"Ya" : 0, "Tidak" : 1}
latihan = {"Rutin" : 0, "Jarang" : 1}
stamina = {"Kuat" : 0, "Lemah" : 1}
mental = {"PD" : 0, "Gerogi" : 1}
menang = {"Ya" : 0, "Tidak" : 1}
dataset["Pelatih"] = dataset["Pelatih"].map(pelatih)
dataset["Kandang Sendiri"] = dataset["Kandang Sendiri"].map(main)
dataset["Latihan"] = dataset["Latihan"].map(latihan)
dataset["Stamina"] = dataset["Stamina"].map(stamina)
dataset["Mental"] = dataset["Mental"].map(mental)
dataset["Menang"] = dataset["Menang"].map(menang)

In [ ]:
dataset.head()

In [ ]:
dataset = dataset.fillna(0)
latihan = dataset['Latihan'].astype(int)
dataset['Latihan'] = latihan

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
x = dataset.iloc[:, 1:6].values #kondisi
y = dataset.iloc[:, -1].values #label

In [ ]:
from sklearn.model_selection import train_test_split

validation_size = 0.20
num_trees = 100
seed = 7
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=validation_size, random_state=seed)

In [ ]:
print("Dataset Asli Menang       : {0} ({1:0.2f}%)".format(len(dataset.loc[dataset['Menang'] == 0]), (len(dataset.loc[dataset['Menang'] == 0])/len(dataset.index)) * 100.0))
print("Dataset Asli Kalah     : {0} ({1:0.2f}%)".format(len(dataset.loc[dataset['Menang'] == 1]), (len(dataset.loc[dataset['Menang'] == 1])/len(dataset.index)) * 100.0))
print("")
print("Dataset Training Menang   : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 0]), (len(y_train[y_train[:] == 0])/len(y_train) * 100.0)))
print("Dataset Training Kalah : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 1]), (len(y_train[y_train[:] == 1])/len(y_train) * 100.0)))
print("")
print("Dataset Test Menang      : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 0]), (len(y_test[y_test[:] == 0])/len(y_test) * 100.0)))
print("Dataset Test Kalah     : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 1]), (len(y_test[y_test[:] == 1])/len(y_test) * 100.0)))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


#Check Algorithms
models = []
models.append(( 'LR' , LogisticRegression()))
models.append(( 'LDA' , LinearDiscriminantAnalysis()))
models.append(( 'KNN' , KNeighborsClassifier()))
models.append(( 'CART' , DecisionTreeClassifier()))
models.append(( 'NB' , GaussianNB()))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed)))
# evaluate each model
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=2, random_state=seed)
  cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring= 'accuracy' )
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

**Algoritma GaussianNB paling bagus : 93++%**

In [ ]:
nb = GaussianNB()
nb.fit(x_train, y_train)
predictions = nb.predict(x_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

dari data test yang di uji, semua nya di prediksi benar dan accuracy nya 100% . GG, mungkin dikarenakan faktor data nya yang bisa dibilang terlalu sedikit

In [ ]:
nb = GaussianNB()
nb.fit(x_train, y_train)
predictions = nb.predict(x_train)
print(accuracy_score(y_train, predictions))
print(confusion_matrix(y_train, predictions))
print(classification_report(y_train, predictions))

# Kita memberikan data kita kepada pak Tatang

Misal pak Tatang, memilih pelatih dan tipe pemain sebagai berikut:
* pelatih : Menekan
* Match : Kandang sendiri
* Latihan : rutin
* Stamina : Kuat karena terbiasa latihan berat
* Mental : grogi , karena dapat tekanan dari pelatih

Kita prediksi hasil akhirnya

In [ ]:
prediction_nb=nb.predict([[1,0,0,0,1]])
if prediction_nb[0] == 0:
    pred = "Menang"
else:
    pred = "Kalah"
print('Prediksi :',pred)

Sayang sekali, Club pak Tatang kalah. maybe next time pak Tatang.

Dikarenakan laga kemaren, pak Tatang mencoba pelatih yang lebih friendly kepada pemain. dengan tipe berikut:

* pelatih : Pengertian
* Match : Away 
* Latihan : Rutin, karena segan dengan pelatih
* Stamina : kuat karena sering latihan
* Mental : Pede , karena mendapatkan support dari pelatih 

Kita prediksi hasil akhirnya

In [ ]:
prediction_nb=nb.predict([[0,1,0,0,0]])
if prediction_nb[0] == 0:
    pred = "Menang"
else:
    pred = "Kalah"
print('Prediksi :',pred)

walaupun pada match ke-2 bermain tandang atau away, tidak mempengaruhi mental pemain menjadi lemah,malah menjadi kiat dikarenakan mendapatkan support system dari pelatih sekarang,yang diamana berpengaruh kepada permainan dari tatang fc.
akhirnya tatang Fc memenangkan pertandingan pertamanya dan pak tatang senang atas kerja yang kita lakukan.